# Nature Poems

In [1]:
import numpy as np
import pandas as pd 
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 1000)
import random
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
import requests as rq

Using TensorFlow backend.


In [2]:
#extract the nature poems from first dataset
df = pd.read_csv('poems.csv') # load dataset
poems = df['content']
naturePoems = poems.loc[df['type'] == 'Nature']

In [3]:
naturePoems.head()

19    Why didst thou promise such a beauteous day,\r...
20    The welcome Sun from sea Freake is returned,\r...
21    I met a courtier riding on the plain,\r\nWell-...
22    Walking the fields a wantcatcher I spied,\r\nT...
23    Fishing, if I a fisher may protest,\r\nOf plea...
Name: content, dtype: object

In [4]:
#convert nature series to a string
naturePoems = naturePoems.to_string()

In [5]:
# MAPPING CHARACTERS: for nature poems, Make two dictionaries , one to convert chars to ints, the other to convert ints back to chars

chars = sorted(list(set(naturePoems)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
# process the dataset:
seqlen = 50
step = seqlen

data_X = []
data_y = []

poemLines = [] 

# creates poem lines 
for i in range(0, len(naturePoems) - seqlen - 1, step):
    poemLines.append(naturePoems[i: i + seqlen + 1])
    
# creating x and y data
data_X = np.zeros((len(poemLines), seqlen, len(chars)), dtype=np.bool)
data_Y = np.zeros((len(poemLines), seqlen, len(chars)), dtype=np.bool)

for i, poemLines in enumerate(poemLines):
    for t, (char_in, char_out) in enumerate(zip(poemLines[:-1], poemLines[1:])):
        data_X[i, t, char_indices[char_in]] = 1
        data_Y[i, t, char_indices[char_out]] = 1

In [8]:
# create the model
model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['categorical_crossentropy', 'accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 128)           104960    
_________________________________________________________________
dense_1 (Dense)              (None, 50, 76)            9804      
Total params: 114,764
Trainable params: 114,764
Non-trainable params: 0
_________________________________________________________________


In [9]:
# train the model
model.fit(data_X, data_Y, batch_size=128, epochs=80)

Epoch 1/80
214/214 [==============================] - 1s 5ms/step - loss: 4.2536 - categorical_crossentropy: 4.2536 - accuracy: 0.0961
Epoch 2/80
214/214 [==============================] - 0s 2ms/step - loss: 4.0437 - categorical_crossentropy: 4.0437 - accuracy: 0.1679
Epoch 3/80
214/214 [==============================] - 1s 2ms/step - loss: 3.3978 - categorical_crossentropy: 3.3978 - accuracy: 0.2180
Epoch 4/80
214/214 [==============================] - 0s 2ms/step - loss: 3.3034 - categorical_crossentropy: 3.3034 - accuracy: 0.2173
Epoch 5/80
214/214 [==============================] - 1s 2ms/step - loss: 3.2562 - categorical_crossentropy: 3.2562 - accuracy: 0.2181
Epoch 6/80
214/214 [==============================] - 0s 2ms/step - loss: 3.2431 - categorical_crossentropy: 3.2431 - accuracy: 0.2190
Epoch 7/80
214/214 [==============================] - 0s 2ms/step - loss: 3.2887 - categorical_crossentropy: 3.2887 - accuracy: 0.2298
Epoch 8/80
214/214 [==============================] - 1

214/214 [==============================] - 0s 1ms/step - loss: 1.6564 - categorical_crossentropy: 1.6564 - accuracy: 0.5179
Epoch 62/80
214/214 [==============================] - 0s 1ms/step - loss: 1.6087 - categorical_crossentropy: 1.6087 - accuracy: 0.5320
Epoch 63/80
214/214 [==============================] - 0s 1ms/step - loss: 1.6244 - categorical_crossentropy: 1.6244 - accuracy: 0.5279
Epoch 64/80
214/214 [==============================] - 0s 1ms/step - loss: 1.6500 - categorical_crossentropy: 1.6500 - accuracy: 0.5275
Epoch 65/80
214/214 [==============================] - 0s 2ms/step - loss: 1.5525 - categorical_crossentropy: 1.5525 - accuracy: 0.5434
Epoch 66/80
214/214 [==============================] - 0s 1ms/step - loss: 1.5473 - categorical_crossentropy: 1.5473 - accuracy: 0.5471
Epoch 67/80
214/214 [==============================] - 0s 1ms/step - loss: 1.5285 - categorical_crossentropy: 1.5285 - accuracy: 0.5511
Epoch 68/80
214/214 [==============================] - 0s 2m

In [10]:
def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)  

In [13]:
def generating_poem(epoch, start_index, diversity, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = start_index
    diversity = diversity

    generated = ''
    poemLines = naturePoems[start_index: start_index + seqlen]
    generated += poemLines
    print('----- Generating with seed: "' + poemLines + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(poemLines):
            x_pred[0, t, char_indices[char]] = 1.
            
        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        poemLines = poemLines[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

In [14]:
generating_poem(80, 50, .25, _)


----- Generating text after Epoch: 80
----- Generating with seed: ",\r...
20     The welcome Sun from sea Freake is r"
,\r...
20     The welcome Sun from sea Freake is rod Ise...
413    When wast ma bear was foor and greass\...
48     Where was a wart mans and sore from Selected Poems...
414    The walle fare was fare fare from Selected Poems...
488    The walle faster fare farthe beace from Selected Poems...
41     The walles and love farmen from Selected Poems...
411                                                                                                
